## Pretprocessing

#### Ucitavamo dataset i pretprocesiramo ga.

In [1]:
import pandas as pd
import spacy
import numpy as np

c:\Users\marin\anaconda3\envs\tarlab1\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\marin\anaconda3\envs\tarlab1\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\marin\anaconda3\envs\tarlab1\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
train_set = pd.read_csv(filepath_or_buffer='./Dataset/train.csv', sep=',')
# test_set = pd.read_csv(filepath_or_buffer='./Dataset/test.csv', sep=',')
# sample_submission_set = pd.read_csv(filepath_or_buffer='./Dataset/sampleSubmission.csv', sep=',')

In [16]:
print(train_set["query"][152])

16 gb memory card


In [17]:
print(train_set["product_description"][152])

SanDisk Elevate 16GB non-HS MSD 3in, microSD memory card with USB adapter and SD adapter. Includes RescuePro recovery software.
Memory Storage Capacity: 16GB
Wired Connectivity: Micro SD Slot
Features: Plug and Play
Includes: MicroSD Adapter, Adapter, USB Adapter
Battery no battery used


In [19]:
print(train_set["product_description"][5])

Durability and all-day comfort of lightweight, stress-resistant O MATTER® frame material
    Integrated surge ports to channel cooling airflow
    Optimized peripheral view and side coverage of Polaric Ellipsoid™ lens geometry
    Comes standard with Oakley Hydrophobic/Oleophobic anti-smudge lens coating
    Comfort and performance of Three-Point Fit that holds lenses in precise optical alignment
    Metal icon accents
    Optical precision and performance that meets all ANSI Z87.1 standards
    Impact resistance that meets all ANSI Z87.1 standards for high-mass and high-velocity impact
    UV protection of PLUTONITE® lens that filters out 100% of UVA / UVB / UVC & harmful blue light up to 400 nm
    Glare reduction and tuned light transmission of IRIDIUM® lens coating
    Multiple interchangeable UNOBTAINIUM® nose pad options for customizable and comfortable secure fit
    Protective sports-specific Oakley Soft Vault included with capacity for an extra lens
    Polished Black frame/Gr

In [18]:
print(len(train_set))

10158


In [23]:
nlp = spacy.load("en_core_web_sm")

In [52]:
doc_product = nlp(train_set["product_description"][152])
doc_query = nlp(train_set["query"][152])
query_tokens = [str(token) for sent in doc_query.sents for token in sent]
print(type(doc_query))
l = list(doc_query)
print(l)
print(type(l[0]))

for sent in doc_product.sents:
    for token in sent:
        #print(token)
        if str(token) in query_tokens:
            print(token)

#print(query_tokens)



<class 'spacy.tokens.doc.Doc'>
[16, gb, memory, card]
<class 'spacy.tokens.token.Token'>
16
memory
card
16


In [53]:
def preprocess_text(df, include_digit = 0):
    #include_digits = 0 --> don't remove any tokens with digits
    #include_digits = 1 --> remove tokens which all characters are digits
    #include_digits = 2 --> remove all tokens which contain digits

    product_all_preprocessed = []
    query_all_preprocessed = []
    
    for idx, row in df.iterrows():
        product_preprocessed = []
        query_preprocessed = []
        product = row['product']
        query = row['query']

        product = ' '.join(product.split())
        query = ' '.join(query.split())
            
        doc_product = nlp(product)
        doc_query = nlp(query)

        query_tokens = [str(token) for sent in doc_query.sents for token in sent]

        for sent in doc_product.sents: 
            for token in sent:
                if not token.is_stop and not token.is_punct:
                    if (include_digit == 0) \
                        or (include_digit == 1 and not all(map(str.isdigit, str(token)))) \
                        or (include_digit == 2 and not any(map(str.isdigit, str(token)))) \
                        or str(token) in query_tokens:
                        product_preprocessed.append(str.lower(token.lemma_))

        for sent in doc_query.sents: 
            for token in sent:
                if not token.is_stop and not token.is_punct:
                    query_preprocessed.append(str.lower(token.lemma_))

                    

        product_all_preprocessed.append(" ".join(map(str, product_preprocessed)))
        query_all_preprocessed.append(" ".join(map(str, query_preprocessed)))
        
    df['query'] = query_all_preprocessed
    df['product'] = product_all_preprocessed
    
    return df

In [ ]:
print(nlp.Defaults.stop_words)

In [21]:
#spajamo naslov i opis produkta u 1 stupac kako bi ih mogli zajedno pretprocesirat
def get_df_for_preprocessing():
    df = pd.DataFrame(train_set)
    df = df.replace(np.nan, '', regex=True)
    df['product'] = df["product_title"] + " " + df["product_description"]

    return df

In [24]:
df_preprocessed = get_df_for_preprocessing()
df_preprocessed = preprocess_text(df_preprocessed, 0)

In [25]:
print(df_preprocessed["product"][152])

sandisk 16 gb non hs msd flash drive 3in1 black sdsdqr-016g t46a sandisk elevate 16 gb non hs msd 3 microsd memory card usb adapter sd adapter include rescuepro recovery software memory storage capacity 16 gb wired connectivity micro sd slot feature plug play include microsd adapter adapter usb adapter battery battery


In [26]:
df_preprocessed.to_csv(r'./Dataset/dataframes/df_preprocessed_with_numbers.csv', encoding='utf-8', header='true')

In [54]:
df_preprocessed_train_set = get_df_for_preprocessing()
df_preprocessed_without_numbers_partialy = preprocess_text(df_preprocessed_train_set, 1)

In [55]:
print(df_preprocessed_without_numbers_partialy["product"][152])

sandisk 16 gb non hs msd flash drive 3in1 black sdsdqr-016g t46a sandisk elevate 16 gb non hs msd microsd memory card usb adapter sd adapter include rescuepro recovery software memory storage capacity 16 gb wired connectivity micro sd slot feature plug play include microsd adapter adapter usb adapter battery battery


In [56]:
df_preprocessed_without_numbers_partialy.to_csv(r'./Dataset/dataframes/df_preprocessed_without_numbers_partially.csv', encoding='utf-8', header='true')

In [57]:
df_preprocessed_train_set = get_df_for_preprocessing()
df_preprocessed_without_all_numbers = preprocess_text(df_preprocessed_train_set, 2)

In [58]:
print(df_preprocessed_without_all_numbers["product"][152])

sandisk 16 gb non hs msd flash drive black sandisk elevate 16 gb non hs msd microsd memory card usb adapter sd adapter include rescuepro recovery software memory storage capacity 16 gb wired connectivity micro sd slot feature plug play include microsd adapter adapter usb adapter battery battery


In [59]:
print(df_preprocessed_without_numbers_partialy["query"][1])

lead christmas light


In [25]:
del df_preprocessed["product_title"]
del df_preprocessed["product_description"]

In [60]:
df_preprocessed_without_all_numbers.to_csv(r'./Dataset/dataframes/df_preprocessed_without_all_tokens_with_numbers.csv', encoding='utf-8', header='true')